<a href="https://colab.research.google.com/github/linyuehzzz/census_privacy/blob/main/topdown_ols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Define strategy queries and strategy matrix

In [23]:
import torch
import numpy as np

# setup parameters
eps = 17.14
delta = 1e-10
n1, n2, n3, n4 = 8, 2, 2, 63        # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)
N = n1 * n2 * n3 * n4         # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)
q1, q2, q3, q4, q6, q7, q8, q9, q10, q11 = [], [], [], [], [], [], [], [], [], []

AA = torch.tensor(range(N))
AA = AA.reshape([n1, n2, n3, n4])

## Q1: TOTAL (1 cell)
q1 = np.ones((1, N))

## Q2: CENRACE (63 cells)
q2 = np.zeros((n4, N))
for x in range(n4):     # race
    hist_idx = torch.flatten(AA[:, :, :, x]).tolist()
    for i in hist_idx:
        q2[x, i] = 1

## Q3: HISPANIC (2 cells)
q3 = np.zeros((n3, N))
for x in range(n3):
    hist_idx = torch.flatten(AA[:, :, x, :]).tolist()
    for i in hist_idx:
        q3[x, i] = 1    

## Q4: VOTINGAGE (2 cells)
q4 = np.zeros((n2, N))
for x in range(n2):
    hist_idx = torch.flatten(AA[:, x, :, :]).tolist()
    for i in hist_idx:
        q4[x, i] = 1 

## Q6: HHGQ (8 cells)
q6 = np.zeros((n1, N))
for x in range(n1):
    hist_idx = torch.flatten(AA[x, :, :, :]).tolist()
    for i in hist_idx:
        q6[x, i] = 1 

## Q7: HISPANIC*CENRACE (126 cells)
q7 = np.zeros((n3*n4, N))
row_idx = 0 
for x in range(n3): 
    for y in range(n4):
        hist_idx = torch.flatten(AA[:, :, x, y]).tolist()
        for i in hist_idx:
            q7[row_idx, i] = 1
        row_idx += 1

## Q8: VOTINGAGE*CENRACE (126 cells)
q8 = np.zeros((n2*n4, N))
row_idx = 0 
for x in range(n2): 
    for y in range(n4):
        hist_idx = torch.flatten(AA[:, x, :, y]).tolist()
        for i in hist_idx:
            q8[row_idx, i] = 1
        row_idx += 1

## Q9: VOTINGAGE*HISPANIC (4 cells)
q9 = np.zeros((n2*n3, N))
row_idx = 0 
for x in range(n2): 
    for y in range(n3):
        hist_idx = torch.flatten(AA[:, x, y, :]).tolist()
        for i in hist_idx:
            q9[row_idx, i] = 1
        row_idx += 1

## Q10: VOTINGAGE*HISPANIC*CENRACE (252 cells)
q10 = np.zeros((n2*n3*n4, N))
row_idx = 0 
for x in range(n2):
    for y in range(n3):
        for z in range(n4):
            hist_idx = torch.flatten(AA[:, x, y, z]).tolist()
            for i in hist_idx:
                q10[row_idx, i] = 1
            row_idx += 1

## Q11: HHGQ*VOTINGAGE*HISPANIC*CENRACE (2,016 cells)
q11 = np.zeros((N, N))
np.fill_diagonal(q11, 1)

## stack all queries -> strategy matrix
A = np.vstack((q1, q2, q3, q4, q6, q7, q8, q9, q10, q11))

print(q1.shape, q2.shape, q3.shape, q4.shape, q6.shape, q7.shape, q8.shape, q9.shape, q10.shape, q11.shape, A.shape)
A

(1, 2016) (63, 2016) (2, 2016) (2, 2016) (8, 2016) (126, 2016) (126, 2016) (4, 2016) (252, 2016) (2016, 2016) (2600, 2016)


array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])